# Inputs
---

In [1]:
import numpy as np, zarr, nrrd
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
import tifffile

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
p = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt186/outputs/'
outdir = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream_lt186_cap_v2'
!mkdir -p $outdir

In [3]:
r1_path = p + '/r1/stitching/export.n5'
r2_path = p + '/r2/stitching/export.n5'
r6_path = p + '/r6/stitching/export.n5'
r7_path = p + '/r7/stitching/export.n5'

r1_zarr = zarr.open(store=zarr.N5Store(r1_path), mode='r')
r2_zarr = zarr.open(store=zarr.N5Store(r2_path), mode='r')
r6_zarr = zarr.open(store=zarr.N5Store(r6_path), mode='r')
r7_zarr = zarr.open(store=zarr.N5Store(r7_path), mode='r')

r1 = r1_zarr['/c3/s4'][...]
r2 = r2_zarr['/c3/s4'][...]
r6 = r6_zarr['/c3/s4'][...]
r7 = r7_zarr['/c3/s4'][...]


In [4]:
np.max(r1), np.max(r2), np.max(r6), np.max(r7)

(61957, 47284, 63646, 56672)

In [5]:
th = np.percentile(r1, 99.9)
r1[r1>th] = th

th = np.percentile(r2, 99.9)
r2[r2>th] = th

th = np.percentile(r6, 99.9)
r6[r6>th] = th

th = np.percentile(r7, 99.9)
r7[r7>th] = th

In [15]:
np.max(r1), np.max(r2), np.max(r6), np.max(r7)

(167, 239, 192, 178)

In [16]:
tifffile.imwrite(outdir+'/r1.tiff', r1)
tifffile.imwrite(outdir+'/r2.tiff', r2)
tifffile.imwrite(outdir+'/r6.tiff', r6)
tifffile.imwrite(outdir+'/r7.tiff', r7)

In [8]:
fix_path = r6_path
mov_path = r7_path
exp_factor = 2  # replace this with the known expansion factor for your sample to use pre-expansion units

s1_scale = [1,2,2] # z,y,x
s3_scale = [4,8,8]
s4_scale = [9,16,16]

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr.attrs.asdict()
fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * s1_scale 
fix_spacing_s3 = fix_spacing * s3_scale 
fix_spacing_s4 = fix_spacing * s4_scale 

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr.attrs.asdict()
mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * s1_scale
mov_spacing_s3 = mov_spacing * s3_scale 
mov_spacing_s4 = mov_spacing * s4_scale 

# print spacings
print(fix_spacing, mov_spacing)
print(fix_spacing_s1, mov_spacing_s1)
print(fix_spacing_s3, mov_spacing_s3)
print(fix_spacing_s4, mov_spacing_s4)

# print shapes
print(fix_zarr['/c3/s0'].shape, mov_zarr['/c3/s0'].shape)
print(fix_zarr['/c3/s3'].shape, mov_zarr['/c3/s3'].shape)
print(fix_zarr['/c3/s4'].shape, mov_zarr['/c3/s4'].shape)

[0.21  0.115 0.115] [0.21  0.115 0.115]
[0.21 0.23 0.23] [0.21 0.23 0.23]
[0.84 0.92 0.92] [0.84 0.92 0.92]
[1.89 1.84 1.84] [1.89 1.84 1.84]
(2498, 7414, 9085) (2296, 7336, 9091)
(624, 926, 1135) (574, 917, 1136)
(277, 463, 567) (255, 458, 568)


# Alignment
---

### global

In [9]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

In [11]:
# define alignment steps
ransac_kwargs = {
    'blob_sizes':[2, 8],
    'cc_radius':12,
    'match_threshold':0.4,
    'nspots':10000,
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}


steps = [
    # ('ransac', ransac_kwargs,),
    ('affine', affine_kwargs,),
]

In [ ]:
%%time

mov = r7
fix = r6

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    # fix_mask=fix_mask,
    # mov_mask=mov_mask,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt(outdir+'/affine_r7tor6.mat', affine)
tifffile.imwrite(outdir+'/affine_r7tor6.tiff', affine_aligned)

LEVEL:  0  ITERATION:  0  METRIC:  -0.20702828664129455
LEVEL:  0  ITERATION:  1  METRIC:  -0.21050120049471488
LEVEL:  0  ITERATION:  2  METRIC:  -0.21362257461918654
LEVEL:  0  ITERATION:  3  METRIC:  -0.21638232703001875
LEVEL:  0  ITERATION:  4  METRIC:  -0.21877146130413483
LEVEL:  0  ITERATION:  5  METRIC:  -0.22180349472718694
LEVEL:  0  ITERATION:  6  METRIC:  -0.2245637878748205
LEVEL:  0  ITERATION:  7  METRIC:  -0.22532319164822634
LEVEL:  0  ITERATION:  8  METRIC:  -0.22489065873602962
LEVEL:  0  ITERATION:  9  METRIC:  -0.22457530125057004
LEVEL:  0  ITERATION:  10  METRIC:  -0.22433057312458915
LEVEL:  0  ITERATION:  11  METRIC:  -0.2242617080907964
LEVEL:  0  ITERATION:  12  METRIC:  -0.22471070755767303
LEVEL:  0  ITERATION:  13  METRIC:  -0.2259517971327905
LEVEL:  0  ITERATION:  14  METRIC:  -0.228216428664988
LEVEL:  0  ITERATION:  15  METRIC:  -0.23135160208775557
LEVEL:  0  ITERATION:  16  METRIC:  -0.23417446328851943
LEVEL:  0  ITERATION:  17  METRIC:  -0.2366981

In [ ]:
affine[:3,3]

In [ ]:
sns.heatmap(affine[:3,:3] - np.eye(3))
plt.show()
sns.heatmap(affine[:3, 3].reshape(-1,1))
plt.show()